In [1]:
import numpy as np

In [6]:
np_data = np.load(
    "/zhangxin11-tos-volc-engine/tyz/WM_dataset/peach_plate_wooden_full/success/success_peach_plate_wooden_proc_0_num_0_seed_0.npz",
    allow_pickle=True,
)
# np_data = np.load(
#     "/iag_ad_01/ad/zhangxin11/tyz/observation-reward-model/data/world_model_rl_dataset/tomato_pick_and_place/success/success_tomato_proc_0_numid_0000_epsid_0.npz",
#     allow_pickle=True,
# )

from transforms3d import euler


def get_tensor(raw_action):
    position = raw_action["end"]["position"].reshape(-1, 3)
    orientation = raw_action["end"]["orientation"].reshape(-1, 4)
    gripper = raw_action["effector"]["position_gripper"]
    action_euler = np.stack([euler.quat2euler(x, "sxyz") for x in orientation])
    actions = np.concatenate((position, action_euler, gripper), axis=-1)
    return actions


actions = get_tensor(np_data["arr_0"].tolist()["action"])
states = get_tensor(np_data["arr_0"].tolist()["state"])

In [7]:
diff = actions - states
diff_per = diff / actions
print("不同维度action和state差值（百分比）的最大值", np.abs(diff_per).max(axis=0))
print("不同维度action和state差值（百分比）的平均值", np.mean(np.abs(diff_per), axis=0))
print("不同维度action和state差值（绝对大小）的最大值", np.abs(diff).max(axis=0))
print("不同维度action和state差值（绝对大小）的平均值", np.mean(np.abs(diff), axis=0))

不同维度action和state差值（百分比）的最大值 [3.72157434e-02 2.77708795e+01 8.79213514e+01 2.00024915e+00
 6.06699612e+01 9.97629013e-01 1.04000000e+00]
不同维度action和state差值（百分比）的平均值 [0.00329015 0.20152364 0.89235759 0.0239365  2.82902419 0.03419239
 0.99493385]
不同维度action和state差值（绝对大小）的最大值 [0.0158138  0.00770814 0.01720185 6.28274011 0.04427574 0.01728406
 1.04      ]
不同维度action和state差值（绝对大小）的平均值 [0.00133257 0.00179137 0.00207168 0.07518776 0.00217567 0.00267385
 0.99493385]


In [8]:
np_data = np.load(
    "/iag_ad_01/ad/zhangxin11/tyz/reward_diffusion_policy_relative/diffusion_policy/data/pickplace_long/seed_0/ep_0/total_steps.npz",
    allow_pickle=True,
)

actions = np_data["action"]
print(
    "预处理后数据action相邻时刻变化的绝对大小的平均值",
    np.mean(np.abs(np.diff(actions, axis=0)), axis=0),
)
print(
    "预处理后数据action相邻时刻变化的绝对大小的最大值",
    np.max(np.abs(np.diff(actions, axis=0)), axis=0),
)

预处理后数据action相邻时刻变化的绝对大小的平均值 [2.07631573e-04 1.13320441e-04 2.25605014e-04 9.48408411e-05
 4.34128923e-04 1.75781874e-04 1.24610592e-02]
预处理后数据action相邻时刻变化的绝对大小的最大值 [0.01562059 0.00567083 0.01829824 0.00890683 0.04206247 0.00421917
 2.        ]
